In [ ]:
import tensorflow as tf
import numpy as np

#with tf.Session() as sess:
#    new_saver = tf.train.import_meta_graph('model_save.ckpt.meta')
#    new_saver.restore(sess, tf.train.latest_checkpoint('./'))


x_data = np.loadtxt('./inputData.txt', delimiter = ',', unpack = True, dtype = 'float32')
x_data = np.transpose(x_data)
y_data = np.loadtxt('./Label.txt', delimiter = ',', unpack = True, dtype = 'float32')
y_data = np.transpose(y_data)

idx_shuffle = np.random.permutation(len(x_data))
x_data = x_data[idx_shuffle]
y_data = y_data[idx_shuffle]

# Features and Labels
X = tf.placeholder(tf.float32) # 1 x 7*3, (x, y, z)
Y = tf.placeholder(tf.float32) # 1 x 2, softmax 

# Network
w1 = tf.Variable(tf.random_normal([21, 128], stddev = .1))
b1 = tf.Variable(tf.zeros([128]))

L1 = tf.add(tf.matmul(X, w1), b1)
L1_act = tf.nn.relu(L1)
L1_do = tf.nn.dropout(L1_act, keep_prob = 0.7)
                 
w2 = tf.Variable(tf.random_normal([128, 128], stddev = .1))
b2 = tf.Variable(tf.zeros([128]))

L2 = tf.add(tf.matmul(L1_do, w2), b2)
L2_act = tf.nn.relu(L2)
L2_do = tf.nn.dropout(L2_act, keep_prob = 0.7)
      
w3 = tf.Variable(tf.random_normal([128, 2], stddev = .1))
b3 = tf.Variable(tf.zeros([2]))
         
m = tf.add(tf.matmul(L2_do, w3), b3)
model = tf.nn.softmax(m)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = model))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(cost)

# Calculate accuracy
correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))


save_file = './train_model.ckpt'
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for (step) in range(10001):
        for (epoch) in range(36):
            s = 100 * epoch
            e = 100 * (epoch + 1)
            sess.run(optimizer, feed_dict = {X:x_data[s:e], Y:y_data[s:e]})
        
        if (step) % 100 == 0:
            c = sess.run(cost, feed_dict = {X:x_data[0:3600], Y:y_data[0:3600]})
            c_test = sess.run(cost, feed_dict = {X:x_data[3600:], Y:y_data[3600:]})
            #l1 = sess.run(model, feed_dict =  {X:x_data[0:10]}) 
            print(step, "   ", c, "   ", c_test)
            #print(l1)

        #tmp = 0.0
        #count = 0

        #if (step) % 100 == 0:
        #    cost_test = sess.run(cost, feed_dict = {X:x_data[750:], Y:y_data[750:]})
        #    if (tmp < cost_test):
        #        count += 1
        #        tmp = cost_test
        #    else :
        #        count = 0

        #if (count > 10):
        #    c = sess.run(cost, feed_dict = {X:x_data[0:750], Y:y_data[0:750]})
        #    c_test = sess.run(cost, feed_dict = {X:x_data[750:], Y:y_data[750:]})
        #    print(step, "   ", c, "   ", c_test)
        #    break

    print('Accuracy:', sess.run(accuracy, feed_dict = {X:x_data, Y:y_data}))
    
    print(sess.run(w1))
    
    saver.save(sess, save_file)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0     0.66878474     0.6776694
